<IMG SRC="https://avatars2.githubusercontent.com/u/31697400?s=400&u=a5a6fc31ec93c07853dd53835936fd90c44f7483&v=4" WIDTH=125 ALIGN="right">


# GIS

This notebook shows how to export model data so it can be viewed in GIS (QGIS or ArcMAP).

### Contents<a name="TOC"></a>
1. [Model types](#modeltypes)
2. [Export vector data](#vectordata)
3. [Export griddata](#rasterdata)
4. [Add symbology (QGIS)](#symbology)

In [1]:
import nlmod
import os
import flopy
import numpy as np
import geopandas as gpd
import xarray as xr
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

import logging
from IPython.display import FileLink, FileLinks

In [2]:
print(f'nlmod version: {nlmod.__version__}')

# toon informatie bij het aanroepen van functies
logging.basicConfig(level=logging.INFO)

nlmod version: 0.2.4b


### [1. Model types](#TOC)<a name="modeltypes"></a>



#### structured grid



In [3]:
model_ws = 'model1'
model_name = 'IJmuiden'

In [4]:
model_ds_struc = xr.load_dataset(os.path.join(model_ws, 'cache', 'full_model_ds.nc'), mask_and_scale=False)
model_ds_struc

<xarray.Dataset>
Dimensions:             (time: 1, x: 100, y: 60, layer: 40)
Coordinates:
  * time                (time) datetime64[ns] 2019-12-31
  * x                   (x) float64 9.505e+04 9.515e+04 ... 1.048e+05 1.05e+05
  * y                   (y) float64 5e+05 4.998e+05 ... 4.942e+05 4.94e+05
  * layer               (layer) object 'AAOP' 'NASC' 'ONAWA' ... 'PZWAz4' 'MSz1'
Data variables: (12/16)
    idomain             (layer, y, x) int32 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    first_active_layer  (y, x) int32 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    bot                 (layer, y, x) float64 -13.07 -13.07 ... -218.2 -218.5
    top                 (y, x) float64 0.0 0.0 0.0 0.0 ... 2.29 1.49 0.5 0.37
    kh                  (layer, y, x) float64 10.0 10.0 10.0 ... 6.1 6.11 6.11
    kv                  (layer, y, x) float64 10.0 10.0 10.0 ... 0.611 0.611
    ...                  ...
    rws_oppwater_area   (y, x) float64 1e+04 1e+04 1e+04 1e+04 ... 0.0 0.0 0.0
    rws_oppwater_cond   (y, x) float64 1e+03 1e+03 1e+03 1e+03 ... 0.0 0.0 0.0
    rws_oppwater_peil   (y, x) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ahn                 (y, x) float64 nan nan nan nan ... 1.971 nan nan 0.2378
    chd                 (layer, y, x) int32 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    recharge            (y, x) float64 0.0006058 0.0006058 ... 0.0007675
Attributes: (12/17)
    model_name:                        IJmuiden
    mfversion:                         mf6
    model_dataset_created_on:          20220510_18:39:42
    exe_name:                          c:\users\oebbe\02_python\nlmod\nlmod\m...
    model_ws:                          model1
    figdir:                            model1\figure
    ...                                ...
    nodata:                            -999
    anisotropy:                        10
    fill_value_kh:                     1.0
    fill_value_kv:                     0.1
    surface_drn_cond:                  1000
    model_dataset_written_to_disk_on:  20220510_18:39:49

In [5]:
# create gisdir
gisdir_struc = os.path.join(model_ws, 'gis')
if not os.path.exists(gisdir_struc):
    os.mkdir(gisdir_struc)

#### vertex grid



In [6]:
model_ws = 'model3'
model_name = 'IJm_planeten'

In [7]:
model_ds_vert = xr.load_dataset(os.path.join(model_ws, 'cache', 'full_model_ds.nc'), mask_and_scale=False)

# get modelgrid
sim = flopy.mf6.MFSimulation.load(sim_name='mfsim.nam', sim_ws=model_ds_vert.model_ws,
                                  load_only=['DISV'])
gwf = sim.get_model(model_ds_vert.model_name)

# get vertices
model_ds_vert['vertices'] = nlmod.mdims.get_vertices(model_ds_vert, modelgrid=gwf.modelgrid)
model_ds_vert

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package disv...
    skipping package npf...
    skipping package ic...
    skipping package oc...
    skipping package ghb...
    skipping package drn...
    skipping package chd...
    skipping package rch...
    skipping package ims6...


<xarray.Dataset>
Dimensions:             (time: 6, icell2d: 6231, iv: 6459, nvert: 7, layer: 40, xy: 2, vert_per_cid: 4)
Coordinates:
  * time                (time) datetime64[ns] 2015-01-01 ... 2015-01-06
    x                   (icell2d) float64 9.505e+04 9.515e+04 ... 1.05e+05
    y                   (icell2d) float64 5e+05 5e+05 ... 4.94e+05 4.94e+05
  * layer               (layer) object 'AAOP' 'NASC' 'ONAWA' ... 'PZWAz4' 'MSz1'
  * xy                  (xy) <U1 'x' 'y'
Dimensions without coordinates: icell2d, iv, nvert, vert_per_cid
Data variables: (12/21)
    xv                  (iv) float64 9.5e+04 9.51e+04 ... 1.049e+05 1.05e+05
    yv                  (iv) float64 5e+05 5e+05 4.999e+05 ... 4.94e+05 4.94e+05
    icvert              (icell2d, nvert) int32 0 1 2 3 0 ... 6457 6356 -1 -1
    idomain             (layer, icell2d) int32 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1
    first_active_layer  (icell2d) int32 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    bot                 (layer, icell2d) float64 -13.07 -13.07 ... -218.2 -218.5
    ...                  ...
    rws_oppwater_peil   (icell2d) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ahn                 (icell2d) float64 nan nan nan nan ... nan nan 0.224
    chd                 (layer, icell2d) int32 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1
    recharge            (icell2d, time) float64 -0.0004 0.0019 ... -0.0001
    rch_name            (icell2d) object 'rch_1' 'rch_1' ... 'rch_2' 'rch_2'
    vertices            (icell2d, vert_per_cid, xy) float64 9.5e+04 ... 4.94e+05
Attributes: (12/18)
    model_name:                        IJm_planeten
    mfversion:                         mf6
    model_dataset_created_on:          20220623_14:55:23
    exe_name:                          c:\users\oebbe\02_python\nlmod\nlmod\m...
    model_ws:                          model3
    figdir:                            model3\figure
    ...                                ...
    nodata:                            -999
    anisotropy:                        10
    fill_value_kh:                     1.0
    fill_value_kv:                     0.1
    surface_drn_cond:                  1000
    model_dataset_written_to_disk_on:  20220623_14:55:27

In [8]:
# create gisdir
gisdir_vert = os.path.join(model_ws, 'gis')
if not os.path.exists(gisdir_vert):
    os.mkdir(gisdir_vert)

### [2. Export vector data](#TOC)<a name="vectordata"></a>



#### structured

In [9]:
# write model data to a geopackage
fname_geopackage = nlmod.visualise.gis.model_dataset_to_vector_file(model_ds_struc, gisdir=gisdir_struc)

# get download link
FileLink(fname_geopackage, result_html_prefix='klik hier om te downloaden -> ')

C:\Users\oebbe\anaconda3\envs\nlmod\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
C:\Users\oebbe\anaconda3\envs\nlmod\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
C:\Users\oebbe\anaconda3\envs\nlmod\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
C:\Users\oebbe\anaconda3\envs\nlmod\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


C:\Users\oebbe\02_python\nlmod\examples\model1\gis\IJmuiden.gpkg

In [10]:
# write model data to multiple shapefiles
fnames = nlmod.visualise.gis.model_dataset_to_vector_file(model_ds_struc, driver='ESRI Shapefile', gisdir=gisdir_struc)

# get download link
FileLinks(gisdir_struc, included_suffixes='.shp')

C:\Users\oebbe\anaconda3\envs\nlmod\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
c:\users\oebbe\02_python\nlmod\nlmod\visualise\gis.py:314: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(fname, driver=driver)
C:\Users\oebbe\anaconda3\envs\nlmod\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
c:\users\oebbe\02_python\nlmod\nlmod\visualise\gis.py:314: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(fname, driver=driver)
C:\Users\oebbe\anaconda3\envs\nlmod\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Ind

model1\gis/
  ahn.shp
  chd.shp
  idomain.shp
  kh.shp
  kv.shp
  recharge.shp
  rws_oppwater.shp
  rws_oppwater_area.shp
  rws_oppwater_cond.shp
  rws_oppwater_peil.shp
  sea.shp
  starting_head.shp
  topbot.shp

#### vertex grid

In [11]:
# write model data to a geopackage
fname_geopackage = nlmod.visualise.gis.model_dataset_to_vector_file(model_ds_vert, gisdir=gisdir_vert)

C:\Users\oebbe\anaconda3\envs\nlmod\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
C:\Users\oebbe\anaconda3\envs\nlmod\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
C:\Users\oebbe\anaconda3\envs\nlmod\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
C:\Users\oebbe\anaconda3\envs\nlmod\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [12]:
# write model data to multiple shapefiles
nlmod.visualise.gis.model_dataset_to_vector_file(model_ds_vert, driver='ESRI Shapefile', gisdir=gisdir_vert)

C:\Users\oebbe\anaconda3\envs\nlmod\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
c:\users\oebbe\02_python\nlmod\nlmod\visualise\gis.py:314: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(fname, driver=driver)
C:\Users\oebbe\anaconda3\envs\nlmod\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
c:\users\oebbe\02_python\nlmod\nlmod\visualise\gis.py:314: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(fname, driver=driver)
C:\Users\oebbe\anaconda3\envs\nlmod\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Ind

['model3\\gis\\idomain.shp',
 'model3\\gis\\topbot.shp',
 'model3\\gis\\sea.shp',
 'model3\\gis\\chd.shp',
 'model3\\gis\\recharge.shp',
 'model3\\gis\\kh.shp',
 'model3\\gis\\ahn.shp',
 'model3\\gis\\rws_oppwater_peil.shp',
 'model3\\gis\\rws_oppwater_cond.shp',
 'model3\\gis\\rws_oppwater_area.shp',
 'model3\\gis\\starting_head.shp',
 'model3\\gis\\kv.shp']

### [3. Export griddata](#TOC)<a name="rasterdata"></a>

The model data can be exported to a netcdf file that can be visualised in Qgis. For a structured model the standard model dataset (xarray.Dataset) can be exported to a netdf file. For a vertex model you have to convert the model dataset to a certain format before you can write it to a netcdf and read it with Qgis. With the code below we export the vertex model dataset to a netcdf file ('model_qgis.nc') that can be read using Qgis. For more background, see this discussion https://github.com/ArtesiaWater/nlmod/issues/14. 

#### structured grid

In order to load a structured grid netcdf file in Qgis use the "Add Raster Layer.." option.

In [18]:
# write model data to a netcdf file
fname = os.path.join(gisdir_struc,'model_struc_qgis.nc')
model_ds_struc.to_netcdf(fname)

# get download link
FileLink(fname, result_html_prefix='klik hier om te downloaden -> ')

C:\Users\oebbe\02_python\nlmod\examples\model1\gis\model_struc_qgis.nc

#### vertex grid

In order to load a vertex netcdf file in Qgis use the "Add Mesh Layer.." option.

In [17]:
# write model data to a netcdf file
fname = os.path.join(gisdir_vert,'model_vert_qgis.nc')
out = nlmod.visualise.gis.model_dataset_to_ugrid_nc_file(model_ds_vert.drop_vars('rch_name'), fname)

# get download link
FileLink(fname, result_html_prefix='klik hier om te downloaden -> ')

C:\Users\oebbe\02_python\nlmod\examples\model3\gis\model_vert_qgis.nc

### [4. Add symbology (QGIS)](#TOC)<a name="symbology"></a>

It is always nice to have automatic symbology for your vector data. Some thoughts:
- QGIS can save symbology of a single shapefile in a .qml file
- In QGIS you can add a .qml file to a geopackage thus saving the symbology to a single file.
- You can create a .qml file in QGIS from existing symbology.
- a .qml file is an xml file so theoretically it is possible to modify a .qml file with Python based on the properties of the data.
- It 



Some limitations of the current gis functions:
- when exporting shapefiles to gis, attributes cannot have names longer
than 10 characters. Now the automatic character shortening of fiona is
used. This is not optimal.
- when exporting data variables with dimension time only the mean values
in time are exported in the shapefile to avoid extremely large files.